In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

nice_colors = {
    "RES": "limegreen",
    "Storage": "darkgrey",
    "Electrolysis": "lightcoral",
    "Transport": "blue",
    "Electrification": "coral",
}

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")

df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"])
    & (df["esc"].isin(["hvdc-to-elec", "pipeline-h2-to-elec"]))
    & (df["scenario"].str.contains(snakemake.wildcards["csp"]))
]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "Baseload",
    #    "daily": "Daily",
    #    "weekly": "Weekly",
    #    "biweekly": "Biweekly",
    #    "monthly": "Month",
    #    "quaterly": "Quarterly",
    "annually": "Annual",
}

# limit to scenarios with desired flexibilities
df = df.loc[df["flexibility"].isin(flexibilities.keys())]

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility, categories=flexibilities.values(), ordered=True
)

df = df.sort_values("flexibility")

demand_in_MWh = df.query("subcategory == 'Total demand'")["value"].unique().item()

df = df.query("category == 'cost'")[["esc", "flexibility", "subcategory", "value"]]

# Get rid of numbers of technologies
df["subcategory"] = df.subcategory.str.replace("\s*\d+\s*", "", regex=True)

aggregate_technologies = {
    "Buffer: electricity demand": "Storage",
    "CCGT (exp)": "H2 gas turbine",
    "CCGT (imp)": "H2 gas turbine",
    "H(g) fill compressor station (exp)": "Transport",
    "H(g) pipeline": "Transport",
    "H(g) pipeline decompressor (imp)": "Transport",
    "H(g) submarine pipeline": "Transport",
    "HVDC inverter pair (exp)": "Transport",
    "HVDC inverter pair (imp)": "Transport",
    "HVDC overhead": "Transport",
    "HVDC submarine": "Transport",
    "Hstorage storing (exp)": "Storage",
    "Hstorage storing (imp)": "Storage",
    "Hstorage unstoring (exp)": "Storage",
    "Hstorage unstoring (imp)": "Storage",
    "battery inverter (charging, exp)": "Storage",
    "battery inverter (charging, imp)": "Storage",
    "battery inverter (discharging, exp)": "Storage",
    "battery inverter (discharging, imp)": "Storage",
    "battery storage": "Storage",
    "clean water tank storage": "Storage",
    "csp-tower": "RES",
    "csp-tower TES": "Storage",
    "csp-tower power block": "RES",
    "electrolysis (exp)": "H2 electrolysis",
    "electrolysis (imp)": "H2 electrolysis",
    "hydrogen storage tank type": "Storage",
    "offwind": "RES",
    "onwind": "RES",
    "seawater desalination (exp)": "H2 electrolysis",
    "solar-utility": "RES",
}

# Replace category names with names/techs to aggregate on, then aggregate
df["technology"] = df["subcategory"].replace(aggregate_technologies)
df = (
    df.groupby(["esc", "flexibility", "technology"])["value"]
    .sum()
    .to_frame()
    .reset_index()
)

# Rearrange for plotting
df = df.pivot(index=["esc", "flexibility"], columns="technology", values="value")

# Which technologies (columns) from dataframe to plot and in which order (bottom to top) in stacked bar plot
technologies_to_plot = [
    "RES",
    "Storage",
    "Transport",
    "H2 electrolysis",
    "H2 gas turbine",
]

# Apply Column/Plot order of technologies
df = df[[t for t in technologies_to_plot if t in df.columns]]

# Up till now cost in total for full demand, present results in share of cost per MWh: Calculate here
df = df / demand_in_MWh

# Correct sort order (HVDC first, then pipeline; Yearly first, then no flex)
df = df.sort_index(ascending=[True, False])

# Plotting
fig, ax = plt.subplots(figsize=(6, 3))

df.plot.bar(stacked=True, ax=ax, width=0.3)

# Seperator between ESCs
ax.plot([1.5] * 2, [0, 250], ls="dashed", color="grey")

ax.text(0.5, 175 + 25 / 2, "HVDC", va="center", ha="center")
ax.text(2.5, 175 + 25 / 2, "H2 Pipeline", va="center", ha="center")

# custom legend
legend = ax.legend(
    handles=ax.get_legend_handles_labels()[0][::-1],  # reverse order to match stacking
    labels=ax.get_legend_handles_labels()[1][::-1],  # reverse order to match stacking
    title=None,
    fontsize="small",
    bbox_to_anchor=(1.0, 0),
    loc="lower left",
)

# ax.set_xlabel("Demand matching")
ax.set_xlabel("")
ax.set_ylabel("Costs [EUR/MWh]")

ax.set_xticklabels(
    ["Annual", "Baseload", "Annual", "Baseload"], rotation=0, fontsize="small"
)
ax.set_yticks(np.arange(0, 250, 50))
ax.set_yticks(np.arange(25, 275, 50), minor=True)
ax.yaxis.grid(which="minor", ls="--")

# Disable verticle grid lines
ax.xaxis.grid(False)

ax.set_ylim(0, 210)  # Magic number 200: Highest value from TN ESC H2 pipeline
ax.set_xlim(-0.5, 3.5)

fig.tight_layout()

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")